## Manually label and upload photos to the database

* **Input:** folder with classifc image classification format:

```
images/
    class_1/
        image_1.jpg
        image_2.jpg
        ...
    class_2/
        image_3.jpeg
        image_4.jpeg
        ...
    ...
```

* **Output:** Labelled images stored in GCP and tracked with Weights & Biases Artifacts. 


In [1]:
# Append the upper level directory to sys
import sys
sys.path.append("..")

import pandas as pd
import numpy as np

from pathlib import Path

In [2]:
# !rm -rf food_photos/*
# !rm -rf _MACOSX
# !rm -rf _MACOSX/*

In [4]:
# !unzip -q 2023-02-15-food_photos.zip

In [3]:
# Get list of all paths in food_photos directory
target_dir = '2023-03-14-food_photos'
path = Path(target_dir)
all_paths = sorted(list(path.glob('*/*.jp*g')))
all_paths[:10]

[PosixPath('2023-03-14-food_photos/apple_custard/65d69064-ac2f-419e-bbcd-a23b7bc5453c.jpeg'),
 PosixPath('2023-03-14-food_photos/apple_red/19966794-7257-4568-be81-ef05666cbb84.jpeg'),
 PosixPath('2023-03-14-food_photos/apple_red/1b51c45e-d711-48b9-aff1-05ea4d7bdc68.jpeg'),
 PosixPath('2023-03-14-food_photos/apple_red/1f013e0b-50c8-4222-b59b-5afe569d060a.jpeg'),
 PosixPath('2023-03-14-food_photos/apple_red/35b33e12-7644-4729-9b01-59f388b075e7.jpeg'),
 PosixPath('2023-03-14-food_photos/apple_red/3e612fdc-6fea-4ea0-b196-728beea40947.jpeg'),
 PosixPath('2023-03-14-food_photos/apple_red/3fb07b94-0909-469f-b9c3-4f6cb7a7b0df.jpeg'),
 PosixPath('2023-03-14-food_photos/apple_red/431c10c1-583d-434b-ba1a-3b3033a39104.jpeg'),
 PosixPath('2023-03-14-food_photos/apple_red/474da89c-bcdc-4d72-96a1-150cae32aafd.jpeg'),
 PosixPath('2023-03-14-food_photos/apple_red/6290a638-d664-42b2-8eb6-93133eb85bc2.jpeg')]

In [4]:
len(all_paths)

190

In [5]:
import uuid

def is_valid_uuid(string):
    try:
        uuid.UUID(string)
        return True
    except ValueError:
        return False

filename = "some-file-name"
if is_valid_uuid(filename):
    print(f"{filename} is a valid UUID")
else:
    print(f"{filename} is not a valid UUID")

some-file-name is not a valid UUID


In [6]:
# Loop through all_paths, if the path is a UUID, pass, if not, rename the file to a UUID
for path in all_paths:
    if not is_valid_uuid(path.stem):
        print(f"[INFO] {path} isn't valud UUID, renaming to UUID...")
        new_name = f"{uuid.uuid4()}.jpeg"
        new_path_name = path.parent / new_name
        path.rename(new_path_name)
        print(f"[INFO] Renamed {path} to {new_path_name}")

# Get list of all paths in food_photos directory
path = Path(target_dir)
all_paths = sorted(list(path.glob('*/*.jpeg')))
all_paths[:10]

[PosixPath('2023-03-14-food_photos/apple_custard/65d69064-ac2f-419e-bbcd-a23b7bc5453c.jpeg'),
 PosixPath('2023-03-14-food_photos/apple_red/19966794-7257-4568-be81-ef05666cbb84.jpeg'),
 PosixPath('2023-03-14-food_photos/apple_red/1b51c45e-d711-48b9-aff1-05ea4d7bdc68.jpeg'),
 PosixPath('2023-03-14-food_photos/apple_red/1f013e0b-50c8-4222-b59b-5afe569d060a.jpeg'),
 PosixPath('2023-03-14-food_photos/apple_red/35b33e12-7644-4729-9b01-59f388b075e7.jpeg'),
 PosixPath('2023-03-14-food_photos/apple_red/3e612fdc-6fea-4ea0-b196-728beea40947.jpeg'),
 PosixPath('2023-03-14-food_photos/apple_red/3fb07b94-0909-469f-b9c3-4f6cb7a7b0df.jpeg'),
 PosixPath('2023-03-14-food_photos/apple_red/431c10c1-583d-434b-ba1a-3b3033a39104.jpeg'),
 PosixPath('2023-03-14-food_photos/apple_red/474da89c-bcdc-4d72-96a1-150cae32aafd.jpeg'),
 PosixPath('2023-03-14-food_photos/apple_red/6290a638-d664-42b2-8eb6-93133eb85bc2.jpeg')]

In [7]:
# Find the number of .jpeg images in each subfolder of food_photos and create a dictionary mapping the subfolder name to the number of images
subfolder_to_num_images = {}
for p in all_paths:
    subfolder = p.parent.stem
    if subfolder in subfolder_to_num_images:
        subfolder_to_num_images[subfolder] += 1
    else:
        subfolder_to_num_images[subfolder] = 1
subfolder_to_num_images

{'apple_custard': 1,
 'apple_red': 15,
 'avocado': 4,
 'bacon': 4,
 'banana': 2,
 'beef_patty': 1,
 'beetroot': 1,
 'black_pepper': 2,
 'bread': 3,
 'broccolini': 2,
 'butter': 3,
 'cabbage_green': 1,
 'cabbage_red': 1,
 'capsicum_green': 1,
 'cashews': 2,
 'celery': 1,
 'cheese': 2,
 'chicken_breast': 1,
 'chicken_stir_fry': 5,
 'chilli': 1,
 'chocolate': 2,
 'coconut': 1,
 'coffee': 2,
 'corn': 1,
 'croissant': 2,
 'cucumber': 3,
 'eggplant': 1,
 'eggs': 2,
 'garlic': 6,
 'ginger': 2,
 'grapefruit': 1,
 'grapes_white': 2,
 'green_beans': 1,
 'guacamole': 3,
 'ice': 1,
 'ice_coffee': 1,
 'kiwi_fruit': 1,
 'lemon': 1,
 'lettuce_iceberg': 1,
 'lime': 3,
 'macadamia': 1,
 'mango': 1,
 'maple_syrup': 1,
 'mushroom_portobello': 1,
 'mushroom_white_cups': 1,
 'nectarine': 3,
 'olives': 1,
 'onion_brown': 2,
 'onion_red': 1,
 'onion_white': 1,
 'orange': 1,
 'orange_juice': 1,
 'papaya': 2,
 'passionfruit': 5,
 'pawpaw': 3,
 'peach': 3,
 'pear': 1,
 'pineapple': 2,
 'plum': 6,
 'popcorn': 2,

In [8]:
len(subfolder_to_num_images)

83

In [9]:
# Get all food_types from all_paths
food_types = sorted(list(set([p.parent.name for p in all_paths])))
food_types

['apple_custard',
 'apple_red',
 'avocado',
 'bacon',
 'banana',
 'beef_patty',
 'beetroot',
 'black_pepper',
 'bread',
 'broccolini',
 'butter',
 'cabbage_green',
 'cabbage_red',
 'capsicum_green',
 'cashews',
 'celery',
 'cheese',
 'chicken_breast',
 'chicken_stir_fry',
 'chilli',
 'chocolate',
 'coconut',
 'coffee',
 'corn',
 'croissant',
 'cucumber',
 'eggplant',
 'eggs',
 'garlic',
 'ginger',
 'grapefruit',
 'grapes_white',
 'green_beans',
 'guacamole',
 'ice',
 'ice_coffee',
 'kiwi_fruit',
 'lemon',
 'lettuce_iceberg',
 'lime',
 'macadamia',
 'mango',
 'maple_syrup',
 'mushroom_portobello',
 'mushroom_white_cups',
 'nectarine',
 'olives',
 'onion_brown',
 'onion_red',
 'onion_white',
 'orange',
 'orange_juice',
 'papaya',
 'passionfruit',
 'pawpaw',
 'peach',
 'pear',
 'pineapple',
 'plum',
 'popcorn',
 'potato_brown',
 'potato_chips',
 'potato_kipfler',
 'potato_purple_sweet',
 'potato_sweet',
 'potato_white_sweet',
 'pumpkin',
 'rambutan',
 'rhubarb',
 'rice_white',
 'rockmelon

## Download original labels from Weights & Biases

In [10]:
# Get config
from configs.default_config import config

args = config
print(args)

namespace(annotations_columns_to_export=['filename', 'image_name', 'class_name', 'label', 'split', 'clear_or_confusing', 'whole_food_or_dish', 'one_food_or_multiple', 'label_last_updated_at', 'label_source', 'image_source'], auto_augment=True, batch_size=128, epochs=10, gs_bucket_name='food_vision_bucket_with_object_versioning', gs_image_storage_path='https://storage.cloud.google.com/food_vision_bucket_with_object_versioning/all_images/', input_size=224, label_smoothing=0.1, learning_rate=0.001, model='coatnext_nano_rw_224', num_to_try_and_autocorrect=1000, num_top_n_preds=5, path_to_gcp_credentials='utils/google-storage-key.json', path_to_label_studio_api_key='utils/label_studio_api_key.json', pretrained=True, seed=42, use_mixed_precision=True, wandb_dataset_artifact='food_vision_199_classes_images:latest', wandb_job_type='', wandb_labels_artifact='food_vision_labels:latest', wandb_model_artifact='trained_model:latest', wandb_project='test_wandb_artifacts_by_reference', wandb_run_note

In [11]:
# Connect to GCP
from utils.gcp_utils import set_gcp_credentials, test_gcp_connection
set_gcp_credentials(path_to_key="../utils/google-storage-key.json")
test_gcp_connection()

[INFO] GCP credentials set!
[INFO] GCP connection successful! Access to GCP for saving/loading data and models available.


In [12]:
import wandb

# Initialize a new run
from utils.wandb_utils import wandb_load_artifact, wandb_download_and_load_labels

notes = f"add {len(all_paths)} manually taken photos to the training dataset"

run = wandb.init(project=args.wandb_project, 
                 job_type=args.wandb_job_type,
                 tags=['manual_photo_upload'],
                 notes=notes)

annotations, class_names, class_dict, reverse_class_dict, labels_path = wandb_download_and_load_labels(wandb_run=run,
wandb_labels_artifact_name=args.wandb_labels_artifact)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mrdbourke. Use `wandb login --relogin` to force relogin


[INFO] Labels directory: ./artifacts/food_vision_labels:v47
[INFO] Labels path: artifacts/food_vision_labels:v47/annotations.csv
[INFO] Working with: 278 classes


/home/daniel/code/nutrify/foodvision/notebooks/../utils/wandb_utils.py:33: DtypeWarning: Columns (5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  annotations = pd.read_csv(labels_path)


In [13]:
len(annotations.label.unique())

278

In [14]:
# Make a copy of the annotations
original_annotations = annotations.copy()

In [15]:
# Get the columns of the annotations
columns_to_create = list(original_annotations.columns)
columns_to_create

['filename',
 'image_name',
 'class_name',
 'label',
 'split',
 'clear_or_confusing',
 'whole_food_or_dish',
 'one_food_or_multiple',
 'label_last_updated_at',
 'label_source',
 'image_source']

In [16]:
# Get all the image paths from food_photos
image_paths = list(Path(target_dir).glob('*/*.jpeg'))
len(image_paths)

190

In [17]:
# Check to see what food_types are in class_names
food_types_in_class_names = [food_type for food_type in food_types if food_type in class_names]

# Check to see what food_types are not in class_names
food_types_not_in_class_names = [food_type for food_type in food_types if food_type not in class_names]
len(food_types_in_class_names)

77

In [18]:
food_types_in_class_names

['apple_red',
 'avocado',
 'bacon',
 'banana',
 'beef_patty',
 'beetroot',
 'bread',
 'broccolini',
 'butter',
 'cabbage_green',
 'cabbage_red',
 'capsicum_green',
 'cashews',
 'celery',
 'cheese',
 'chicken_breast',
 'chilli',
 'chocolate',
 'coconut',
 'coffee',
 'corn',
 'croissant',
 'cucumber',
 'eggplant',
 'eggs',
 'garlic',
 'ginger',
 'grapefruit',
 'grapes_white',
 'green_beans',
 'guacamole',
 'ice_coffee',
 'kiwi_fruit',
 'lemon',
 'lettuce_iceberg',
 'lime',
 'macadamia',
 'mango',
 'maple_syrup',
 'mushroom_portobello',
 'mushroom_white_cups',
 'nectarine',
 'olives',
 'onion_brown',
 'onion_red',
 'onion_white',
 'orange',
 'orange_juice',
 'papaya',
 'passionfruit',
 'pawpaw',
 'peach',
 'pear',
 'pineapple',
 'plum',
 'popcorn',
 'potato_brown',
 'potato_chips',
 'potato_kipfler',
 'potato_purple_sweet',
 'potato_sweet',
 'pumpkin',
 'rhubarb',
 'rice_white',
 'rockmelon',
 'salsa',
 'salt',
 'sandwich',
 'silverbeet',
 'squash_butternut',
 'steak',
 'tea',
 'tomato',


In [19]:
food_types_not_in_class_names

['apple_custard',
 'black_pepper',
 'chicken_stir_fry',
 'ice',
 'potato_white_sweet',
 'rambutan']

In [20]:
# Check to see if "macadamia" in class_names
[class_name for class_name in class_names if "pis" in class_name]

['pistachio']

In [21]:
# How to slot new classes into existing class_dict?
# Then update the existing annotations with the new class labels (e.g. "apple_green" -> 2, "apple_red" -> 3)

# Add new classes to class_dict
for food_type in food_types_not_in_class_names:
    class_dict[len(class_dict)] = food_type
    class_names.append(food_type)
    reverse_class_dict[food_type] = len(class_dict)-1

In [22]:
len(class_dict)

284

In [23]:
from utils.misc import get_now_time

# Create a list of dictionaries and fill out the columns of the annotations
columns = ['filename',
 'image_name',
 'class_name',
 'label',
 'split',
 'clear_or_confusing',
 'whole_food_or_dish',
 'one_food_or_multiple',
 'label_last_updated_at',
 'label_source',
 'image_source']

food_image_dict_list = []

LABEL_SOURCE = "manual_upload"
IMAGE_SOURCE = "manual_upload"
LABEL_LAST_UPDATED_AT = get_now_time()

for image_path in image_paths:
    food_image_dict = {}
    food_image_dict['filename'] = image_path
    food_image_dict['image_name'] = image_path.name
    food_image_dict['class_name'] = image_path.parent.name
    food_image_dict['label'] = reverse_class_dict[image_path.parent.name]
    # food_image_dict['split'] = 'train'
    # Label 20% of the images as test
    if np.random.random() < 0.2:
        food_image_dict['split'] = 'test'
    else:
        food_image_dict['split'] = 'train'
    food_image_dict['clear_or_confusing'] = 'clear'
    food_image_dict['whole_food_or_dish'] = 'whole_food'
    food_image_dict['one_food_or_multiple'] = 'one_food'
    food_image_dict['label_last_updated_at'] = LABEL_LAST_UPDATED_AT
    food_image_dict['label_source'] = LABEL_SOURCE
    food_image_dict['image_source'] = IMAGE_SOURCE

    food_image_dict_list.append(food_image_dict)

# Create a dataframe from the list of dictionaries
import pandas as pd
new_annotations = pd.DataFrame(food_image_dict_list)
new_annotations.head()

,filename,image_name,class_name,label,split,clear_or_confusing,whole_food_or_dish,one_food_or_multiple,label_last_updated_at,label_source,image_source
0,2023-03-14-food_photos/watermelon/7e7adb76-a8e...,7e7adb76-a8ed-4f2d-b408-07468854ff63.jpeg,watermelon,272,test,clear,whole_food,one_food,2023-03-14_10-25-03,manual_upload,manual_upload
1,2023-03-14-food_photos/watermelon/e1c1c319-486...,e1c1c319-4862-439b-95d6-81694afddaa1.jpeg,watermelon,272,test,clear,whole_food,one_food,2023-03-14_10-25-03,manual_upload,manual_upload
2,2023-03-14-food_photos/salt/36bab3c9-2ec0-41a2...,36bab3c9-2ec0-41a2-a733-e13d883e7e44.jpeg,salt,232,test,clear,whole_food,one_food,2023-03-14_10-25-03,manual_upload,manual_upload
3,2023-03-14-food_photos/salt/f061158a-352c-4471...,f061158a-352c-4471-8cb7-71d3209ee5f2.jpeg,salt,232,train,clear,whole_food,one_food,2023-03-14_10-25-03,manual_upload,manual_upload
4,2023-03-14-food_photos/chicken_stir_fry/db1912...,db1912c2-b1f3-48fa-bb51-d5b199ebd2d8.jpeg,chicken_stir_fry,280,train,clear,whole_food,one_food,2023-03-14_10-25-03,manual_upload,manual_upload


In [24]:
# Print how many images are train/test in new_annotations
new_annotations['split'].value_counts()

train    154
test      36
Name: split, dtype: int64

In [25]:
# Find a random sample of 10 images where the label value is 199 or higher
new_annotations[new_annotations['label'] >= 199].sample(10)

,filename,image_name,class_name,label,split,clear_or_confusing,whole_food_or_dish,one_food_or_multiple,label_last_updated_at,label_source,image_source
47,2023-03-14-food_photos/rice_white/2e9a2530-bed...,2e9a2530-bed0-49e6-90a3-cffed479297d.jpeg,rice_white,223,train,clear,whole_food,one_food,2023-03-14_10-25-03,manual_upload,manual_upload
8,2023-03-14-food_photos/chicken_stir_fry/1e81fe...,1e81fe96-0999-4497-8cb4-d0b5ce2d5e36.jpeg,chicken_stir_fry,280,train,clear,whole_food,one_food,2023-03-14_10-25-03,manual_upload,manual_upload
43,2023-03-14-food_photos/rice_white/02330fc6-aad...,02330fc6-aada-4ab8-93b1-58a194d1f093.jpeg,rice_white,223,test,clear,whole_food,one_food,2023-03-14_10-25-03,manual_upload,manual_upload
29,2023-03-14-food_photos/zucchini/589fce5d-8592-...,589fce5d-8592-4359-a97f-56f7a2f6d7d9.jpeg,zucchini,277,test,clear,whole_food,one_food,2023-03-14_10-25-03,manual_upload,manual_upload
36,2023-03-14-food_photos/zucchini/3cc5deab-0526-...,3cc5deab-0526-47d6-8cdf-d8b1392bf611.jpeg,zucchini,277,test,clear,whole_food,one_food,2023-03-14_10-25-03,manual_upload,manual_upload
7,2023-03-14-food_photos/chicken_stir_fry/6caaf9...,6caaf985-4eac-470e-9dbf-2d983d29afe9.jpeg,chicken_stir_fry,280,train,clear,whole_food,one_food,2023-03-14_10-25-03,manual_upload,manual_upload
44,2023-03-14-food_photos/rice_white/1731b41c-d1e...,1731b41c-d1ec-4781-b830-268f4f99a9d0.jpeg,rice_white,223,train,clear,whole_food,one_food,2023-03-14_10-25-03,manual_upload,manual_upload
114,2023-03-14-food_photos/potato_sweet/71a490ca-7...,71a490ca-7c47-48a2-8450-69c21999328c.jpeg,potato_sweet,209,train,clear,whole_food,one_food,2023-03-14_10-25-03,manual_upload,manual_upload
117,2023-03-14-food_photos/potato_purple_sweet/f3a...,f3a55544-dddd-4839-986a-2b893782f286.jpeg,potato_purple_sweet,205,test,clear,whole_food,one_food,2023-03-14_10-25-03,manual_upload,manual_upload
161,2023-03-14-food_photos/potato_chips/88f799be-b...,88f799be-b80d-41d4-805c-744e9db1584c.jpeg,potato_chips,202,train,clear,whole_food,one_food,2023-03-14_10-25-03,manual_upload,manual_upload


In [26]:
len(original_annotations), len(new_annotations)

(132198, 190)

In [27]:
# Append the new_annotations to the original_annotations
updated_annotations = pd.concat([original_annotations, new_annotations], ignore_index=True)

# updated_annotations = original_annotations.(new_annotations, ignore_index=True)
updated_annotations

,filename,image_name,class_name,label,split,clear_or_confusing,whole_food_or_dish,one_food_or_multiple,label_last_updated_at,label_source,image_source
0,test/pain_au_chocolat/4fd7cb42-bd7f-48f1-bfdc-...,4fd7cb42-bd7f-48f1-bfdc-607c2f54b788.jpg,pain_au_chocolat,170,test,NaN,NaN,NaN,NaN,NaN,internet_download
1,test/pain_au_chocolat/2062f52a-781c-4e4f-b8a7-...,2062f52a-781c-4e4f-b8a7-0a108934f453.jpg,pain_au_chocolat,170,test,NaN,NaN,NaN,NaN,NaN,internet_download
2,test/pain_au_chocolat/8003e0f6-37e8-460d-9c14-...,8003e0f6-37e8-460d-9c14-e7c6fe44a37f.jpg,pain_au_chocolat,170,test,NaN,NaN,NaN,NaN,NaN,internet_download
3,test/pain_au_chocolat/839437c8-c643-408f-9f04-...,839437c8-c643-408f-9f04-d0d3bec238c3.jpg,pain_au_chocolat,170,test,NaN,NaN,NaN,NaN,NaN,internet_download
4,test/pain_au_chocolat/ca5c13ff-a535-4b69-9144-...,ca5c13ff-a535-4b69-9144-e06275e01e35.jpg,pain_au_chocolat,170,test,NaN,NaN,NaN,NaN,NaN,internet_download
...,...,...,...,...,...,...,...,...,...,...,...
132383,2023-03-14-food_photos/guacamole/16007af8-6b4b...,16007af8-6b4b-483e-8e8a-87cd864a8ec4.jpeg,guacamole,108,test,clear,whole_food,one_food,2023-03-14_10-25-03,manual_upload,manual_upload
132384,2023-03-14-food_photos/guacamole/369cdf67-4382...,369cdf67-4382-4b1f-b9ab-d7f206c0a8a4.jpeg,guacamole,108,test,clear,whole_food,one_food,2023-03-14_10-25-03,manual_upload,manual_upload
132385,2023-03-14-food_photos/ice_coffee/2e466ef8-809...,2e466ef8-8090-429f-80be-43682da2abae.jpeg,ice_coffee,116,train,clear,whole_food,one_food,2023-03-14_10-25-03,manual_upload,manual_upload
132386,2023-03-14-food_photos/onion_brown/08cc18d2-62...,08cc18d2-6217-49a6-b41e-01614e3dc5a5.jpeg,onion_brown,162,train,clear,whole_food,one_food,2023-03-14_10-25-03,manual_upload,manual_upload


In [28]:
# How many unique class_names are in original_annotations?
len(original_annotations['class_name'].unique())

278

In [29]:
# How many unique class_names are in updated_annotations?
len(updated_annotations['class_name'].unique())

284

In [30]:
from typing import List, Dict, Tuple

def get_updated_class_names_class_dict_and_reverse_class_dict(df: pd.DataFrame) -> Tuple[List[str], Dict[int, str], Dict[str, int]]:
    """Get an updated class names list, class dict and reverse class dict from a dataframe."""
    updated_class_names = sorted(df.class_name.unique())
    updated_class_dict = {i: class_name for i, class_name in enumerate(updated_class_names)}
    updated_reverse_class_dict = {class_name: i for i, class_name in enumerate(updated_class_names)}
    assert len(updated_class_dict) == len(updated_reverse_class_dict), "Class dict and reverse class dict are not the same length"
    return updated_class_names, updated_class_dict, updated_reverse_class_dict

def map_updated_class_dict_to_updated_annotations(df: pd.DataFrame, updated_reverse_class_dict: Dict[str, int]) -> pd.DataFrame:
    """Map updated class dict to updated annotations dataframe.
    
    For example, go from {"apple_red": 1} -> df["label"] = 1
    """
    updated_annotations = df.copy()
    updated_annotations.loc[:, 'label'] = updated_annotations['class_name'].map(updated_reverse_class_dict)
    assert len(updated_annotations.label.unique()) == len(updated_reverse_class_dict), "Number of unique labels in updated annotations does not match number of unique class names in updated reverse class dict"
    return updated_annotations

In [31]:
updated_class_names, updated_class_dict, updated_reverse_class_dict = get_updated_class_names_class_dict_and_reverse_class_dict(updated_annotations)

updated_annotations = map_updated_class_dict_to_updated_annotations(updated_annotations, updated_reverse_class_dict)

len(updated_class_names), updated_class_names[:10]

(284,
 ['almond_butter',
  'almonds',
  'apple_custard',
  'apple_green',
  'apple_red',
  'apricot',
  'artichoke',
  'asparagus',
  'avocado',
  'bacon'])

In [33]:
# Check the difference in lengths between the original_annotations and updated_annotations
from utils.misc import check_for_differences_between_df
num_differences = check_for_differences_between_df(original_annotations, updated_annotations)
# num_differences = len(updated_annotations) - len(original_annotations)
num_differences

Number of intersecting columns: 11
Checking for differences accross the following columns: ['filename', 'image_name', 'class_name', 'label', 'split', 'clear_or_confusing', 'whole_food_or_dish', 'one_food_or_multiple', 'label_last_updated_at', 'label_source', 'image_source']
Error: Can only compare identically-labeled DataFrame objects
Couldn't compare via pandas.DataFrame.compare, trying via lengths...
Difference in dataframe lengths: 190 (aboslute value of 132198 - 132388)


190

In [34]:
# Upload the updated annotations to Google Storage and track the changes
from utils.gcp_utils import upload_to_gs, rename_blob, delete_blob
from utils.wandb_utils import wandb_add_artifact_with_reference
from utils.misc import get_now_time
import os

GS_BUCKET_NAME = config.gs_bucket_name

UPDATED_ANNOTATIONS_TARGET_FILENAME = "updated_annotations.csv"
ORIGINAL_ANNOTATIONS_TARGET_FILENAME = "annotations.csv"

# Export the updated annotations to a CSV
columns_to_export = config.annotations_columns_to_export
print(f"[INFO] Exporting the following columns to {UPDATED_ANNOTATIONS_TARGET_FILENAME}: {columns_to_export}")

# TODO: Check if the updated_annotations_reset_index and the original_annotations actually differ, if so save them and upload them, else exit
if num_differences > 0:
    print(f"[INFO] {num_differences} changes to annotations.csv, updated label files and original annotations are different, saving the updated annotations.csv")

    # Export the updated_annotations_reset_index to a csv
    updated_annotations[columns_to_export].to_csv(UPDATED_ANNOTATIONS_TARGET_FILENAME, index=False)

    # Upload the updated CSV to Google Storage
    upload_to_gs(bucket_name=GS_BUCKET_NAME, 
                 source_file_name=UPDATED_ANNOTATIONS_TARGET_FILENAME, 
                 destination_blob_name=UPDATED_ANNOTATIONS_TARGET_FILENAME)

    # Rename the old CSV on Google Storage
    bucket_to_move_old_annotations_to = "old_annotations"
    name_to_rename_old_annotations = os.path.join(bucket_to_move_old_annotations_to, f"{get_now_time()}_old_annotations.csv")

    rename_blob(bucket_name=GS_BUCKET_NAME,
                blob_name=ORIGINAL_ANNOTATIONS_TARGET_FILENAME,
                new_name=name_to_rename_old_annotations)

    # Rename the "updated_annotations.csv" on Google Storage to "annotations.csv" 
    rename_blob(bucket_name=GS_BUCKET_NAME,
                blob_name=UPDATED_ANNOTATIONS_TARGET_FILENAME,
                new_name=ORIGINAL_ANNOTATIONS_TARGET_FILENAME)

    # Track the changes in the annotations with Weights & Biases
    annotations_path_on_gcs = f"gs://{GS_BUCKET_NAME}/{ORIGINAL_ANNOTATIONS_TARGET_FILENAME}"
    wandb_add_artifact_with_reference(wandb_run=run,
                                      artifact_name="food_vision_labels",
                                      artifact_type="labels",
                                      description="Labels for FoodVision project",
                                      reference_path=annotations_path_on_gcs)
else:
    print("[INFO] No changes to annotations.csv, updated label files and original annotations are the same, try fixing/updating the label files and try again")

[INFO] Exporting the following columns to updated_annotations.csv: ['filename', 'image_name', 'class_name', 'label', 'split', 'clear_or_confusing', 'whole_food_or_dish', 'one_food_or_multiple', 'label_last_updated_at', 'label_source', 'image_source']
[INFO] 190 changes to annotations.csv, updated label files and original annotations are different, saving the updated annotations.csv
[INFO] Uploading updated_annotations.csv to updated_annotations.csv...
[INFO] Connected to Google Storage bucket: food_vision_bucket_with_object_versioning
[INFO] File updated_annotations.csv uploaded to food_vision_bucket_with_object_versioning/updated_annotations.csv.
[INFO] File size: 26443341 bytes
[INFO] Blob annotations.csv has been renamed to old_annotations/2023-03-14_10-26-19_old_annotations.csv
[INFO] Blob updated_annotations.csv has been renamed to annotations.csv
[INFO] Logging 'food_vision_labels' from 'gs://food_vision_bucket_with_object_versioning/annotations.csv' to Weights & Biases...


## Upload all photos in food_photos to Google Storage

In [35]:
from utils.gcp_utils import upload_to_gs, get_list_of_blobs

# Get list of images already in GCP bucket
gs_image_paths = get_list_of_blobs(bucket_name=GS_BUCKET_NAME, prefix="all_images")
print(f"[INFO] There are {len(gs_image_paths)} images in the GCP bucket")

[INFO] There are 139522 images in the GCP bucket


In [36]:
gs_image_paths_name = [str(blob.name).split("/")[-1] for blob in gs_image_paths]
gs_image_paths_name[:10]

['000000000.jpg',
 '000000003.jpg',
 '000000005.jpg',
 '000000006.jpg',
 '000000008.jpg',
 '000000010.jpg',
 '000000015.jpg',
 '000000019.jpg',
 '000000025.jpg',
 '000000070.jpg']

In [37]:
len(gs_image_paths_name)

139522

In [38]:
print(target_dir)

2023-03-14-food_photos


In [39]:
# Upload the images to Google Storage
!gsutil -m cp -r $target_dir/*/*.jp*g gs://$GS_BUCKET_NAME/all_images/

# Make sure all photos are in top level directory of all_images/
!gsutil -m mv gs://$GS_BUCKET_NAME/all_images/*/* gs://$GS_BUCKET_NAME/all_images/

Copying file://2023-03-14-food_photos/apple_custard/65d69064-ac2f-419e-bbcd-a23b7bc5453c.jpeg [Content-Type=image/jpeg]...
Copying file://2023-03-14-food_photos/apple_red/19966794-7257-4568-be81-ef05666cbb84.jpeg [Content-Type=image/jpeg]...
Copying file://2023-03-14-food_photos/apple_red/1b51c45e-d711-48b9-aff1-05ea4d7bdc68.jpeg [Content-Type=image/jpeg]...
Copying file://2023-03-14-food_photos/apple_red/1f013e0b-50c8-4222-b59b-5afe569d060a.jpeg [Content-Type=image/jpeg]...
Copying file://2023-03-14-food_photos/apple_red/35b33e12-7644-4729-9b01-59f388b075e7.jpeg [Content-Type=image/jpeg]...
Copying file://2023-03-14-food_photos/apple_red/3e612fdc-6fea-4ea0-b196-728beea40947.jpeg [Content-Type=image/jpeg]...
Copying file://2023-03-14-food_photos/apple_red/3fb07b94-0909-469f-b9c3-4f6cb7a7b0df.jpeg [Content-Type=image/jpeg]...
Copying file://2023-03-14-food_photos/apple_red/431c10c1-583d-434b-ba1a-3b3033a39104.jpeg [Content-Type=image/jpeg]...
Copying file://2023-03-14-food_photos/apple_

In [40]:
# Get list of images already in GCP bucket
gs_image_paths = get_list_of_blobs(bucket_name=GS_BUCKET_NAME, prefix="all_images")
print(f"[INFO] There are {len(gs_image_paths)} images in the GCP bucket")

[INFO] There are 139712 images in the GCP bucket


In [42]:
gs_image_paths_name = [str(blob.name).split("/")[-1] for blob in gs_image_paths]
gs_image_paths_name[:10]

['000000000.jpg',
 '000000003.jpg',
 '000000005.jpg',
 '000000006.jpg',
 '000000008.jpg',
 '000000010.jpg',
 '000000015.jpg',
 '000000019.jpg',
 '000000025.jpg',
 '000000070.jpg']

In [43]:
# Track the changes to GCP bucket with Weights & Biases
from utils.wandb_utils import wandb_add_artifact_with_reference

In [44]:
# Track updated images_dir in Weights & Biases
food_images_path_on_gs = "gs://food_vision_bucket_with_object_versioning/all_images"

wandb_add_artifact_with_reference(wandb_run=run,
                                  artifact_name="food_vision_199_classes_images",
                                  artifact_type="dataset",
                                  description="Images for FoodVision project",
                                  reference_path=food_images_path_on_gs)

[INFO] Logging 'food_vision_199_classes_images' from 'gs://food_vision_bucket_with_object_versioning/all_images' to Weights & Biases...


wandb: Generating checksum for up to 1000000000 objects with prefix "all_images"... Done. 31.2s


In [45]:
# Get list of images already in GCP bucket
gs_image_paths = get_list_of_blobs(bucket_name=GS_BUCKET_NAME, prefix="all_images")
print(f"[INFO] There are {len(gs_image_paths)} images in the GCP bucket")

[INFO] There are 139712 images in the GCP bucket


In [62]:
gs_image_paths[0].name

'all_images/000226a7-5332-4f45-b0e9-6760e9bd6d3e.jpeg'

In [ ]:
# Next:
# Create labels CSV for all photos in food_photos ✅
# Add labels CSV to original annotations ✅
# Upload all photos in food_photos to GCP ✅
# Track updates in W&B ✅

# Train a model and track how it performs (by only upgrading training data)